# TCR Library Design

In [170]:
import pandas as pd
import cfg

from Bio import Seq
from Bio import SeqIO


In [171]:
#Import reference sequence as DNA and AA seq objects

REF_DNA = SeqIO.read("data/TRBC_Extracellular.fasta", "fasta").seq
REF_AA = Seq.translate(REF_DNA)

BLOSSOM_SUB_TABLE = pd.read_csv('data/blosum80_subs_plus_a.csv') # Dataframe with list of substitutions for a given amino acid based on blossom matrix

In [172]:
def homologous_sub_freq(): #Generate dictionary with ranked list of substitutions at a given position
    homology_dt = pd.read_csv('data/homologous_subs.csv') #DT with sub frequencies from alignment
    sub_freq_dict = {} #Dictionary with pre-sorted list of subs at a given position

    aa_row = list(homology_dt["AA"]) #List of which row corresponds to which AA

    for position in homology_dt.keys()[1:]: #Iterates through sequence but skips first column with AA table
        position_subs = list(homology_dt[position]) #Frequencies of all subs (index matches aa_row)
        sub_tuples = [] #List of formatted sub frequencies (sub aa, frequency)

        for i in range(len(position_subs)): #Iterates through list of sub frequencies and reformats them into tuples

            aa = aa_row[i] #sub aa
            sub_freq = position_subs[i] #frequency in homologous sequences

            sub_tuples += [(aa, sub_freq)]

        subs_by_freq = sorted(sub_tuples, key=lambda x: x[1], reverse=True) #Sorted list of subs
        sub_freq_dict[int(position)] = subs_by_freq

    return sub_freq_dict

def load_top_codons(): #Generate dictionary of top 2 codons for each AA
    aa_codon_dt = pd.read_csv('data/hs_codon_freq.csv') #Dataframe of codon frequencies
    top_2_codons_dt = aa_codon_dt.groupby(['aa'])['codon','freq'].apply(lambda x: x.nlargest(2, columns=['freq']))
    top_codons = {} #Dictionary with list of top two codons for a given AA

    for codon in top_2_codons_dt.iterrows():
        aa = codon[0][0]
        codon_dna = codon[1][0]

        if aa in top_codons.keys():
            top_codons[aa] += [codon_dna]

        else:
            top_codons[aa] = [codon_dna]

    return top_codons

TOP_2_CODONS = load_top_codons()
HOMOLOGY_SUB_TABLE = homologous_sub_freq()


<ipython-input-172-1c63247b486b>:25: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  top_2_codons_dt = aa_codon_dt.groupby(['aa'])['codon','freq'].apply(lambda x: x.nlargest(2, columns=['freq']))


## Mutation Generation:

Mutations are generated by iterating through the amino acid sequence of the protein. The first 4 substitutions for a
given position are alanine and the top 3 results from the blossom table. I then plan to look for the most frequent
mutation seen at that position in homologous sequences that is not included in the intial set of substitutions.

This list of AA substititions will be fed to Dan's code to pick the top two codons for that AA, formatted into a
dictionary that includes the position mutated, the original AA at that position, the new AA at that position, the
new codon used, the new DNA sequence, and the new AA sequence.

These dictionaries

In [173]:
def delete_position(ref_dna, ref_aa, aa_i):
    '''

    :param ref_dna:
    :param ref_aa:
    :param aa_i:
    :return:
    '''

    dna_i = aa_i * 3

    deleted_position = {
        "position": aa_i,
        "original_aa": ref_aa[aa_i],
        "new_aa": "*",
        "new_codon": "",
        "new_dna_seq": ref_dna[:dna_i] + ref_dna[dna_i + 3:],
        "new_aa_seq": ref_aa[:aa_i] + ref_aa[aa_i+1:]
    }

    return [deleted_position]

def generate_subs(ref_dna, ref_aa, aa_i):
    '''
    Generates substitute sequence dictionary entries for a given position to alanine, the top 4 blossom substitutions,
    and any potential substitutions found in homologous sequences

    :param ref_dna: DNA sequence being mutated
    :param ref_aa:  AA sequence being mutated
    :param aa_i: Position which is being mutated
    :return: List of dictionary entries containing details of mutations
    '''

    unique_subs_count = 0

    dna_i = aa_i * 3
    aa = ref_aa[aa_i] #AA being substituted

    sub_aas = [] #List of new AAs
    sub_sequences = [] #List of mutation dictionaries

    blossom_subs = list(BLOSSOM_SUB_TABLE[aa]) #Alanine + top 4 mutations from blossom table
    sub_aas += blossom_subs


    if aa_i >= 2:
        for sub, freq in HOMOLOGY_SUB_TABLE[aa_i]:
            if sub not in sub_aas and freq > 0:
                sub_aas += [sub]
                unique_subs_count += 1


    for sub_aa in sub_aas:
        codons = AA_TO_CODON[sub_aa]

        for codon in codons:
            sub_sequence = {
                "position": aa_i,
                "original_aa": ref_aa[aa_i],
                "new_aa": sub_aa,
                "new_codon": codon,
                "new_dna_seq": ref_dna[:dna_i] + codon + ref_dna[dna_i + 3:],
                "new_aa_seq": ref_aa[:aa_i] + sub_aa + ref_aa[aa_i+1:]
            }

            sub_sequences += [sub_sequence]

    return sub_sequences, unique_subs_count

def generate_library(ref_dna, ref_aa):
    '''

    :param ref_dna:
    :param ref_aa:
    :return:
    '''


    count = 0
    mutated_sequences = []

    for i in range(len(REF_AA)):
        mutated_sequences += delete_position(ref_dna, ref_aa, i)
        mutated_sequences += generate_subs(ref_dna, ref_aa, i)[0]

    count += generate_subs(ref_dna, ref_aa, i)[1]

    return pd.DataFrame(mutated_sequences)

mutated_sequences = generate_library(REF_DNA, REF_AA)
mutated_sequences.to_csv('out/results.csv')

MUTATED_SEQUENCES = mutated_sequences

## Testing:
Basic sanity checks for a given mutation:

1.) Does the translated new DNA sequence match the new AA sequence?

2.) Is the new DNA sequence different from the reference DNA sequence?

3.) Does the new codon match the new AA?

4.) Does the new AA match the

In [174]:
def check_translation(mutation_dict):
    if Seq.translate(mutation_dict["new_dna_seq"]) != mutation_dict["new_aa_seq"]:
        return False

    return True

def check_dna_change(mutation_dict):
    if mutation_dict["new_dna_seq"] == REF_DNA:
        return False
    return True

#for sequence in MUTATED_SEQUENCES:
#    print(check_dna_change(sequence))
#    print(check_translation(sequence))



